# New dataset

In [2]:
%pip install torch -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.2 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.18.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.2 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [4]:
#Change the seeds
def xorshift128():
    '''xorshift
    https://en.wikipedia.org/wiki/Xorshift
    '''

    x = 12354
    y = 23412
    z = 79827
    w = 4129

    def _random():
        nonlocal x, y, z, w
        t = x ^ ((x << 11) & 0xFFFFFFFF)  # 32bit
        x, y, z = y, z, w
        w = (w ^ (w >> 19)) ^ (t ^ (t >> 8))
        return w

    return _random

x = open('xorshift128.txt','w')
r = xorshift128()

for i in range(4000000):
    _ = x.write(str(r())+'\n')


In [5]:
IMPORT_COUNT = 1990000
TEST_COUNT = 10000

In [6]:
# Generate random seed
#myrand=np.random.randint(1, 99999 + 1)
myrand=71926
np.random.seed(myrand)
torch.manual_seed(myrand)
# tf.random.set_seed(myrand)
print("Random seed is:",myrand)

Random seed is: 71926


In [7]:
def number_to_binary_string(n):
    return format(n if n >= 0 else (1 << BIT_WIDTH) + n, '032b')

# Convert binary string to array of bits
def binary_string_to_bit_array(binary_string):
    return np.array([int(bit) for bit in binary_string], dtype=int)

def binary_sequence_to_number(binary_sequence):
    return sum(bit * (2 ** idx) for idx, bit in enumerate(binary_sequence[::-1]))

In [15]:
PREVIOUS_TIMESTEP_COUNT = 4
TOTAL_DATA_NUM = IMPORT_COUNT-PREVIOUS_TIMESTEP_COUNT

In [16]:
# convert the sequence of generated numbers to 4 inputs and one output
def strided(a, L):
	shp = a.shape
	s  = a.strides
	nd0 = shp[0]-L+1
	shp_in = (nd0,L)+shp[1:]
	strd_in = (s[0],) + s
	return np.lib.stride_tricks.as_strided(a, shape=shp_in, strides=strd_in)

In [17]:
RNG_OUTPUT_FILENAME="xorshift128.txt"
df = np.genfromtxt(RNG_OUTPUT_FILENAME,delimiter='\n',dtype='uint64')[:IMPORT_COUNT]

In [18]:
# calculates how many bits are in the output.
BIT_WIDTH = np.ceil(np.log2(np.amax(df))).astype(int)

In [19]:
df_as_strings = [number_to_binary_string(number) for number in df]
df_as_bits = np.vstack([binary_string_to_bit_array(binary_string) for binary_string in df_as_strings])
df_as_frames = strided(df_as_bits, PREVIOUS_TIMESTEP_COUNT+1)

In [20]:
indicies = np.arange(TOTAL_DATA_NUM,dtype='uint64')
np.random.shuffle(indicies)
df_as_frames=df_as_frames[indicies]

In [21]:
# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])

In [22]:
# Convert the data into train and test data
X_train = X[TEST_COUNT:]
X_test = X[:TEST_COUNT]
y_train = y[TEST_COUNT:]
y_test = y[:TEST_COUNT]

In [23]:
X_train.shape, y_train.shape

((1979996, 128), (1979996, 32))

## Load model with the initial sequence and predict the next

In [24]:
class XORShift128Model(nn.Module):
    def __init__(self):
        super(XORShift128Model, self).__init__()
        self.fc1 = nn.Linear(128, 1024)  # First dense layer
        self.fc2 = nn.Linear(1024, 32)   # Second dense layer, output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        # x = torch.sigmoid(self.fc2(x))  # Applying sigmoid to ensure output is between 0 and 1
        return x

# Initialize the model
model = XORShift128Model()

# Print the model structure
print(model)

XORShift128Model(
  (fc1): Linear(in_features=128, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=32, bias=True)
)


In [25]:
# Define a custom dataset class
class XORDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [26]:
train_data = XORDataset(X_train,y_train)
test_data = XORDataset(X_test, y_test)

In [27]:
batch_size = 512  # Adjust this size to your needs
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_data, batch_size=512, shuffle=False)

In [28]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Training on device: {device}.")

Training on device: cuda.


## Set up Criteria

In [29]:
criterion = nn.BCEWithLogitsLoss()
## These are taking from the source code
hp = {
    'learning_rate': 0.00038,
    'epsilon': 6.4e-07,
    'beta_1': 0.85,
    'beta_2': 0.88
}

# Configure the optimizer with these hyperparameters
optimizer = torch.optim.NAdam(model.parameters(), 
                              lr=hp['learning_rate'], 
                              betas=(hp['beta_1'], hp['beta_2']),
                              eps=hp['epsilon'])
model = model.to(device)

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_data_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)  
        loss = criterion(outputs, targets)  
        loss.backward()
        optimizer.step()

    # Epoch end
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 0.5909
Epoch [2/100], Loss: 0.5203
Epoch [3/100], Loss: 0.5188
Epoch [4/100], Loss: 0.5073
Epoch [5/100], Loss: 0.4734
Epoch [6/100], Loss: 0.4471
Epoch [7/100], Loss: 0.4241
Epoch [8/100], Loss: 0.3849
Epoch [9/100], Loss: 0.3400
Epoch [10/100], Loss: 0.3073
Epoch [11/100], Loss: 0.2810
Epoch [12/100], Loss: 0.2637
Epoch [13/100], Loss: 0.2407
Epoch [14/100], Loss: 0.2196
Epoch [15/100], Loss: 0.1971
Epoch [16/100], Loss: 0.1802
Epoch [17/100], Loss: 0.1734
Epoch [18/100], Loss: 0.1701
Epoch [19/100], Loss: 0.1681
Epoch [20/100], Loss: 0.1607
Epoch [21/100], Loss: 0.1413
Epoch [22/100], Loss: 0.1221
Epoch [23/100], Loss: 0.1073
Epoch [24/100], Loss: 0.0968
Epoch [25/100], Loss: 0.0889
Epoch [26/100], Loss: 0.0819
Epoch [27/100], Loss: 0.0749
Epoch [28/100], Loss: 0.0659
Epoch [29/100], Loss: 0.0629
Epoch [30/100], Loss: 0.0624
Epoch [31/100], Loss: 0.0620
Epoch [32/100], Loss: 0.0617
Epoch [33/100], Loss: 0.0614
Epoch [34/100], Loss: 0.0611
Epoch [35/100], Loss: 0

## Save Model

In [31]:
torch.save(model.state_dict(), "pytorch_model_new.pth")